In [1]:
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

In [2]:
from sqlalchemy import create_engine

POSTGRES_ADDRESS = '192.168.160.111'

POSTGRES_PORT = '5432' 

POSTGRES_USERNAME = 'postgres'

POSTGRES_PASSWORD = 'jabicopostgres112000'

POSTGRES_DBNAME = 'tagleaf_lims'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
               .format(username=POSTGRES_USERNAME,
                password=POSTGRES_PASSWORD,
                ipaddress=POSTGRES_ADDRESS,
                port=POSTGRES_PORT,
                dbname=POSTGRES_DBNAME))
#Create the connection

cnx = create_engine(postgres_str)

<h5>Create dataframe from SQL query</h5>

In [3]:
base_df = pd.read_sql_query('''SELECT co.compliance_version, co.sample_submission_id, s.name, s.client_id, 
c.organization_name, s.sample_type_id, s.properties, co.coa_data, co.properties from clients c 
join sample_submissions s on c.id=s.client_id join coas co on s.id=co.sample_submission_id 
where c.id not in (1, 7, 17) and co.deleted_at is null order by s.sample_type_id;''', cnx)

<h5>Rename columns to accomodate having two columns with the name 'properties'</h5>

In [4]:
base_df.columns = ['compliance_version', 'sample_submission_id', 'name', 'client_id', 'organization_name', 'sample_type_id', 'ss_properties', 'coa_data', 'coa_properties']
base_df.head()

,compliance_version,sample_submission_id,name,client_id,organization_name,sample_type_id,ss_properties,coa_data,coa_properties
0,2018-07-01,3446,18020-A-RE,70,"Aureus DHS, LLC",1,{'tests_info': {'tests': {'cannabinoid_test': ...,"{'pesticide_test': {'base_test_type_id': 6, 't...",None
1,2018-07-01,3447,18018-N-RE,70,"Aureus DHS, LLC",1,{'tests_info': {'tests': {'terpenoid_test': [{...,"{'pesticide_test': {'base_test_type_id': 6, 't...",None
2,2019-01-17,17003,BB2T,396,Abstrax Labs,1,"{'compliance_type': 'adult_use', 'tests_info':...","{'heavy_metal_test': [{'base_test_type_id': 7,...",{'report_dry_weight': False}
3,2019-01-17,17719,HFD,560,3247 AM LLC. (Dab Daddy),1,"{'compliance_type': 'adult_use', 'tests_info':...","{'cannabinoid_test': [{'base_test_type_id': 3,...",{'report_dry_weight': False}
4,2018-07-01,4207,IH091103,130,Loud Pack QA,1,{'tests_info': {'tests': {'pesticide_test': [{...,"{'pesticide_test': {'base_test_type_id': 6, 't...",None


<h5>Create masks for selecting each kind of coa test result set </h5>

In [5]:
cannabinoid_test_list = []
terpenoid_test_list = []
pesticide_test_list = []
residual_solvent_test_list = []
heavy_metal_test_list = []
microbial_test_list = []
mycotoxin_test_list = []
foreign_material_test_list = []
homogeneity_test_list = []
moisture_test_list = []
water_activity_test_list = []

for index, row in base_df.iterrows():
    #print(row)
    cannabinoid_test_list.append('cannabinoid_test' in row['coa_data'])
    terpenoid_test_list.append('terpenoid_test' in row['coa_data'])
    pesticide_test_list.append('pesticide_test' in row['coa_data'])
    residual_solvent_test_list.append('residual_solvent_test' in row['coa_data'])
    heavy_metal_test_list.append('heavy_metal_test' in row['coa_data'])
    microbial_test_list.append('microbial_test' in row['coa_data'])
    mycotoxin_test_list.append('mycotoxin_test' in row['coa_data'])
    foreign_material_test_list.append('foreign_material_test' in row['coa_data'])
    homogeneity_test_list.append('homogeneity_test' in row['coa_data'])
    moisture_test_list.append('moisture_test' in row['coa_data'])
    water_activity_test_list.append('water_activity_test' in row['coa_data'])
    
cannabinoid_test_mask = pd.Series(cannabinoid_test_list)
terpenoid_test_mask = pd.Series(terpenoid_test_list)
pesticide_test_mask = pd.Series(pesticide_test_list)
residual_solvent_test_mask = pd.Series(residual_solvent_test_list)
heavy_metal_test_mask = pd.Series(heavy_metal_test_list)
microbial_test_mask = pd.Series(microbial_test_list)
mycotoxin_test_mask = pd.Series(mycotoxin_test_list)
foreign_material_test_mask = pd.Series(foreign_material_test_list)
homogeneity_test_mask = pd.Series(homogeneity_test_list)
moisture_test_mask = pd.Series(moisture_test_list)
water_activity_test_mask = pd.Series(water_activity_test_list)

base_df['has_cannabinoid_test'] = (cannabinoid_test_mask)
base_df['has_terpenoid_test'] = (terpenoid_test_mask)
base_df['has_pesticide_test'] = (pesticide_test_mask)
base_df['has_residual_solvent_test'] = (residual_solvent_test_mask)
base_df['has_heavy_metal_test'] = (heavy_metal_test_mask)
base_df['has_microbial_test'] = (microbial_test_mask)
base_df['has_mycotoxin_test'] = (mycotoxin_test_mask)
base_df['has_foreign_material_test'] = (foreign_material_test_mask)
base_df['has_homogeneity_test'] = (homogeneity_test_mask)
base_df['has_moisture_test'] = (moisture_test_mask)
base_df['has_water_activity_test'] = (water_activity_test_mask)

<h5>Save this as dataframe as the base dataframe</h5>

In [6]:
base_df.to_pickle('./pkls/base_coa_data.pkl')

<h1> Everything after this point needs to be redone </h1>

In [7]:
base_df.describe()

,sample_submission_id,client_id,sample_type_id
count,20418.000000,20418.000000,20418.000000
mean,13454.599520,294.816192,4.338819
std,7416.258724,218.365817,4.068607
min,2.000000,2.000000,1.000000
25%,6787.250000,70.000000,1.000000
50%,13928.500000,295.000000,4.000000
75%,20081.750000,485.000000,6.000000
max,25338.000000,798.000000,18.000000


In [11]:
base_df.groupby(['organization_name', 'name']).count().nlargest(20, 'compliance_version')


compliance_version  \
organization_name                        name                                       
CA Cannabis Factory QA                   RAW DISTILLATE                       110   
CONNECTED MANAGEMENT LLC                 AREA 41                               33   
Aureus DHS, LLC                          SKYWALKER                             28   
                                         WAUI                                  28   
                                         COOKIES                               26   
                                         KUSH                                  26   
                                         OG                                    26   
CONNECTED MANAGEMENT LLC                 BISCOTTI                              26   
Aureus DHS, LLC                          GLUE                                  23   
                                         TANGIE                                22   
TOLUCA LAKE COLLECTIVE / TLC JUNGLE BOYS TOPANGA CANYON OG                     21   
Aureus DHS, LLC                          APE                                   20   
                                         DREAM                                 20   
                                         HAZE                                  19   
City Compassionate Caregivers            MONSTER COOKIES                       19   
Aureus DHS, LLC                          SHERBET                               18   
City Compassionate Caregivers            MIMOSA                                18   
TOLUCA LAKE COLLECTIVE / TLC JUNGLE BOYS JUNGLE CAKE                           18   
3247 AM LLC. (Dab Daddy)                 FINISHED DISTILLATE                   16   
                                         MAIN BODY DISTILLATE                  16   

                                                               sample_submission_id  \
organization_name                        name                                         
CA Cannabis Factory QA                   RAW DISTILLATE                         110   
CONNECTED MANAGEMENT LLC                 AREA 41                                 33   
Aureus DHS, LLC                          SKYWALKER                               28   
                                         WAUI                                    28   
                                         COOKIES                                 26   
                                         KUSH                                    26   
                                         OG                                      26   
CONNECTED MANAGEMENT LLC                 BISCOTTI                                26   
Aureus DHS, LLC                          GLUE                                    23   
                                         TANGIE                                  22   
TOLUCA LAKE COLLECTIVE / TLC JUNGLE BOYS TOPANGA CANYON OG                       21   
Aureus DHS, LLC                          APE                                     20   
                                         DREAM                                   20   
                                         HAZE                                    19   
City Compassionate Caregivers            MONSTER COOKIES                         19   
Aureus DHS, LLC                          SHERBET                                 18   
City Compassionate Caregivers            MIMOSA                                  18   
TOLUCA LAKE COLLECTIVE / TLC JUNGLE BOYS JUNGLE CAKE                             18   
3247 AM LLC. (Dab Daddy)                 FINISHED DISTILLATE                     16   
                                         MAIN BODY DISTILLATE                    16   

                                                               client_id  \
organization_name                        name                              
CA Cannabis Factory QA                   RAW DISTILLATE              110   
CONNECTED MANAGEMENT LLC                 AREA 41 

In [18]:
df_coa_data.columns

Index(['coa_data', 'has_cannabinoid_test', 'has_terpenoid_test',
       'has_pesticide_test', 'has_residual_solvent_test',
       'has_heavy_metal_test', 'has_microbial_test', 'has_mycotoxin_test',
       'has_foreign_material_test', 'has_homogeneity_test',
       'has_moisture_test', 'has_water_activity_test',
       'has_cannabinoid_and_terpenoid_test',
       'has_heavy_metal_and_terpenoid_test'],
      dtype='object')

In [8]:
df_pesticide = df_coa_data[(df_coa_data['has_pesticide_test'] == True)]
print(df_pesticide.shape)
df_pesticide.to_pickle("./pkls/has_pesticide.pkl")

(13036, 14)


In [9]:
df_cannabinoid = df_coa_data[(df_coa_data['has_cannabinoid_test'] == True)]
print(df_cannabinoid.shape)
df_cannabinoid.to_pickle("./pkls/has_cannabinoid.pkl")

(13476, 14)


In [6]:
df_terpenoid = df_coa_data[(df_coa_data['has_terpenoid_test'] == True)]
print(df_terpenoid.shape)
df_terpenoid.to_pickle("./pkls/has_terpenoid.pkl")

(1819, 14)


In [7]:
df_residual_solvent = df_coa_data[(df_coa_data['has_residual_solvent_test'] == True)]
print(df_residual_solvent.shape)
df_residual_solvent.to_pickle("./pkls/has_residual_solvent.pkl")

(3918, 14)


In [10]:
df_heavy_metal = df_coa_data[(df_coa_data['has_heavy_metal_test'] == True)]
print(df_heavy_metal.shape)
df_heavy_metal.to_pickle("./pkls/has_heavy_metal.pkl")

(6460, 14)


In [13]:
df_microbial = df_coa_data[(df_coa_data['has_microbial_test'] == True)]
print(df_microbial.shape)
df_microbial.to_pickle("./pkls/has_microbial.pkl")

(7627, 14)


In [15]:
df_mycotoxin = df_coa_data[(df_coa_data['has_mycotoxin_test'] == True)]
print(df_mycotoxin.shape)
df_mycotoxin.to_pickle("./pkls/has_mycotoxin.pkl")

(5406, 14)


In [18]:
df_foreign_material = df_coa_data[(df_coa_data['has_foreign_material_test'] == True)]
print(df_foreign_material.shape)
df_foreign_material.to_pickle("./pkls/has_foreign_material.pkl")

(6636, 14)


In [19]:
df_homogeneity = df_coa_data[(df_coa_data['has_homogeneity_test'] == True)]
print(df_homogeneity.shape)
df_homogeneity.to_pickle("./pkls/has_homogeneity.pkl")

(18, 14)


In [20]:
df_moisture = df_coa_data[(df_coa_data['has_moisture_test'] == True)]
print(df_moisture.shape)
df_moisture.to_pickle("./pkls/has_moisture.pkl")

(4862, 14)


In [21]:
df_water_activity = df_coa_data[(df_coa_data['has_water_activity_test'] == True)]
print(df_water_activity.shape)
df_water_activity.to_pickle("./pkls/has_water_activity.pkl")

(3053, 14)


In [22]:
df_cannabinoid_and_terpene = df_coa_data[(df_coa_data['has_cannabinoid_and_terpenoid_test'])]
print(df_cannabinoid_and_terpene.shape)
df_cannabinoid_and_terpene.to_pickle("./pkls/has_cannabinoid_and_terpenoid")

(1704, 14)


In [23]:
df_heavy_metal_and_terpene = df_coa_data[(df_coa_data['has_heavy_metal_and_terpenoid_test'])]
print(df_heavy_metal_and_terpene.shape)
df_heavy_metal_and_terpene.to_pickle("./pkls/has_heavy_metal_and_terpenoid")

(1479, 14)


<h5>Get more information from the COA</h5>

In [30]:
df = pd.read_sql_query('''SELECT id, coa_data FROM coas where deleted_at is null;''', cnx)

In [29]:
df.head()

,id,coa_data
0,38517,{'foreign_material_test': [{'test_name': 'BCL-...
1,38387,{'cannabinoid_test': [{'test_name': 'BCL-03: C...
2,5109,{'pesticide_test': {'test_name': 'BCL-05: Resi...
3,26098,{'mycotoxin_test': [{'test_name': 'BCL-11: Myc...
4,31848,{'foreign_material_test': [{'test_name': 'BCL-...


In [31]:
pesticide_test_list = []
for index, row in df.iterrows():
    pesticide_test_list.append('pesticide_test' in row[1])
pesticide_test_mask = pd.Series(pesticide_test_list)
df['has_pesticide_test'] = (pesticide_test_mask)
print(df.head())

      id                                           coa_data  \
0  20844  {'moisture_test': [{'test_name': 'BCL-02: Mois...   
1  39389  {'cannabinoid_test': [{'test_name': 'BCL-03: C...   
2  21151  {'foreign_material_test': [{'test_name': 'BCL-...   
3   7768  {'pesticide_test': {'test_name': 'BCL-05: Resi...   
4   7989  {'pesticide_test': {'test_name': 'BCL-05: Resi...   

   has_pesticide_test  
0               False  
1               False  
2                True  
3                True  
4                True  


In [32]:
df_pest = df[(df['has_pesticide_test'] == True)]

In [33]:
df_pest.shape

(13036, 3)

In [34]:
df_pest.to_pickle('./pkls/has_pesticide2.pkl')

<h1>Lets look at Sample Submission data</h1>

In [39]:
df_sample_submission_data = pd.read_sql_query('''SELECT * FROM sample_submissions WHERE deleted_at IS NULL;''', cnx)


In [40]:
df_sample_submission_data

,id,client_id,name,batch_code,lot_num,sample_type_id,received_at,batch_quantity,batch_quantity_type_id,sampled_quantity,...,serving_size_description,status_updated_at,notes,description,package_size,package_size_unit_id,sent_for_review,due_at,sample_manifest_id,submission_type_id
0,21514,296,8,None,None,1,2019-10-25 01:31:31,2,38,2,...,None,2019-10-27 23:04:57,None,None,NaN,NaN,2019-10-25 13:00:04,2019-10-28 19:00:00,None,1
1,3388,192,CBD TINCTURE,None,None,12,2018-08-20 07:00:00,1,33,1,...,1 bottle,2018-10-21 18:37:11,None,None,NaN,NaN,2018-08-20 20:29:42,NaT,None,1
2,3446,70,18020-A-RE,None,None,1,2018-08-21 07:00:00,2,33,2,...,None,2018-10-29 21:44:32,None,None,NaN,NaN,2018-08-21 23:46:15,NaT,None,1
3,17456,70,RM 1058,None,None,1,2019-08-05 20:35:07,2,33,2,...,None,2019-10-15 16:18:20,None,None,NaN,NaN,2019-08-06 13:00:02,2019-08-12 19:00:00,None,1
4,1566,97,PACIFIC OG,None,None,6,2018-07-18 00:00:00,5,38,5,...,None,2018-09-21 21:20:54,None,None,NaN,NaN,2018-07-18 19:57:46,NaT,None,1
5,21933,724,ISOLATE #7B,None,None,1,2019-11-01 15:05:00,1,33,1,...,None,2019-11-05 17:29:30,"POTENCY OUTSIDE CAL CURVE, REDILUTE - MRH 1911...",None,NaN,NaN,2019-11-03 14:00:04,2019-11-09 20:00:00,None,1
6,3447,70,18018-N-RE,None,None,1,2018-08-21 07:00:00,2,33,2,...,None,2018-10-29 21:44:48,None,None,NaN,NaN,2018-08-21 23:46:31,NaT,None,1
7,16589,238,BROWNIES,None,None,4,2019-07-17 17:28:00,20,33,20,...,1/10 OF A BROWNIE,2019-09-17 22:39:21,None,None,79.0,38.0,2019-07-18 13:00:04,2019-07-20 19:00:00,None,1
8,1567,97,SUPER LEMON HAZE,None,None,6,2018-07-18 00:00:00,5,38,5,...,None,2018-09-21 21:20:54,None,None,NaN,NaN,2018-07-18 19:58:12,NaT,None,1
9,1568,97,Larry OG,None,None,6,2018-07-18 00:00:00,5,38,5,...,None,2018-09-21 21:20:54,None,None,NaN,NaN,2018-07-18 19:59:09,NaT,None,1


In [7]:
df_test = pd.read_sql_query('''SELECT co.compliance_version, co.sample_submission_id, s.name, s.client_id, 
c.organization_name, s.sample_type_id, s.properties, co.coa_data from clients c 
join sample_submissions s on c.id=s.client_id join coas co on s.id=co.sample_submission_id 
where c.id not in (1, 7, 17) and co.deleted_at is null order by s.sample_type_id;''', cnx)

In [6]:
#df_test = pd.read_sql_query('''select c.id, co.*, s.sample_type_id, c.organization_name from clients c 
#join sample_submissions s on c.id=s.client_id join coas co on s.id=co.sample_submission_id
#where c.id !=17 and s.sample_type_id not in(16, 17, 18, 19, 20) and co.deleted_at is null order by s.sample_type_id;''', cnx)
 

In [97]:
df_test.head(20)

,compliance_version,sample_submission_id,name,client_id,organization_name,sample_type_id,properties,coa_data
0,2018-07-01,3447,18018-N-RE,70,"Aureus DHS, LLC",1,{'tests_info': {'tests': {'terpenoid_test': [{...,{'cannabinoid_test': {'results': {'CBGA': {'an...
1,2019-01-17,16158,APE,70,"Aureus DHS, LLC",1,"{'label_values': [], 'compliance_type': 'adult...",{'heavy_metal_test': [{'results': {'Cadmium': ...
2,2019-01-17,17013,LB0037,485,TOLUCA LAKE COLLECTIVE / TLC JUNGLE BOYS,1,"{'label_values': [], 'compliance_type': 'adult...",{'heavy_metal_test': [{'results': {'Cadmium': ...
3,2018-07-01,1578,LEMON MINT,6,Pure CA LLC DBA Moxie,1,"{'thc_cbd_ratio': None, 'tests_ordered': 'CA C...",{'cannabinoid_test': {'results': {'CBGA': {'an...
4,2019-01-17,17415,CALIFORNIACITRUS_FG,418,DOM PEN,1,"{'label_values': [], 'compliance_type': 'adult...",{'heavy_metal_test': [{'results': {'Cadmium': ...
5,2019-01-17,11832,AVDOC1020RSS,445,AVD PRO,1,"{'matrix_subtype': None, 'review_sent_by': 'Di...",{'heavy_metal_test': [{'results': {'Cadmium': ...
6,2019-01-17,12179,NORMAL 2.0 UNCAPPED,445,AVD PRO,1,"{'matrix_subtype': None, 'review_sent_by': 'Di...",{'heavy_metal_test': [{'results': {'Cadmium': ...
7,2019-01-17,21474,BISCOTTI CRUMBLE,697,FOG CITY EXTRACTS,1,"{'increments': None, 'compliance_type': 'adult...",{'heavy_metal_test': [{'results': {'Cadmium': ...
8,2019-01-17,16411,FLOOR 2- EXHAUST VENT,296,JB LOS ANGELES,1,"{'label_values': [], 'compliance_type': 'adult...",{'pesticide_test': [{'results': {'Methyl parat...
9,2019-01-17,23047,BZ #004 DE-CARB,104,City Compassionate Caregivers,1,"{'label_values': [], 'increments': None, 'comp...",{'pesticide_test': [{'results': {'Methyl parat...


In [72]:
pd.Index(df_test['compliance_version']).value_counts()

2019-01-17    11730
2018-07-01     5706
2018-12-31      531
2018-01-01      400
Name: compliance_version, dtype: int64

In [94]:
df_test.groupby(['client_id', 'sample_type_id']).count().nlargest(20, 'compliance_version')
#df_test.groupby(['client_id', 'sample_type_id']).count()

,,compliance_version,sample_submission_id,coa_data,properties,name
client_id,sample_type_id,,,,,
70,1,2197,2197,2197,2197,2197
130,1,586,586,586,586,586
529,6,496,496,496,496,496
104,6,368,368,368,368,368
296,1,347,347,347,347,347
83,1,287,287,287,287,287
461,1,272,272,272,272,272
97,6,251,251,251,251,251
309,1,223,223,223,223,223


In [90]:
pd.Index(df_test['name']).value_counts()

RAW DISTILLATE                110
PURPLE PUNCH                   73
WEDDING CAKE                   60
GELATO                         60
BLUE DREAM                     55
OG                             51
SKYWALKER                      49
SOUR DIESEL                    43
GORILLA GLUE                   38
STRAWBERRY BANANA              38
TANGIE                         36
OG KUSH                        35
MIMOSA                         34
DISTILLATE                     34
COOKIES                        33
JACK HERER                     31
SUNSET SHERBERT                27
AREA 41                        27
FIRE OG                        26
PINEAPPLE EXPRESS              26
WAUI                           24
MENDO BREATH                   24
GLUE                           24
BISCOTTI                       24
GG4                            23
KUSH                           23
FD                             22
MONSTER COOKIES                20
KING LOUIE                     20
CBD ISOLATE   

<h5>Need to creat new mask for test</h5>

In [8]:
cannabinoid_test_list = []
terpenoid_test_list = []
pesticide_test_list = []
residual_solvent_test_list = []
heavy_metal_test_list = []
microbial_test_list = []
mycotoxin_test_list = []
foreign_material_test_list = []
homogeneity_test_list = []
moisture_test_list = []
water_activity_test_list = []

for index, row in df_test.iterrows():
    #print(row)
    cannabinoid_test_list.append('cannabinoid_test' in row['coa_data'])
    terpenoid_test_list.append('terpenoid_test' in row['coa_data'])
    pesticide_test_list.append('pesticide_test' in row['coa_data'])
    residual_solvent_test_list.append('residual_solvent_test' in row['coa_data'])
    heavy_metal_test_list.append('heavy_metal_test' in row['coa_data'])
    microbial_test_list.append('microbial_test' in row['coa_data'])
    mycotoxin_test_list.append('mycotoxin_test' in row['coa_data'])
    foreign_material_test_list.append('foreign_material_test' in row['coa_data'])
    homogeneity_test_list.append('homogeneity_test' in row['coa_data'])
    moisture_test_list.append('moisture_test' in row['coa_data'])
    water_activity_test_list.append('water_activity_test' in row['coa_data'])


In [9]:
cannabinoid_test_mask = pd.Series(cannabinoid_test_list)
terpenoid_test_mask = pd.Series(terpenoid_test_list)
pesticide_test_mask = pd.Series(pesticide_test_list)
residual_solvent_test_mask = pd.Series(residual_solvent_test_list)
heavy_metal_test_mask = pd.Series(heavy_metal_test_list)
microbial_test_mask = pd.Series(microbial_test_list)
mycotoxin_test_mask = pd.Series(mycotoxin_test_list)
foreign_material_test_mask = pd.Series(foreign_material_test_list)
homogeneity_test_mask = pd.Series(homogeneity_test_list)
moisture_test_mask = pd.Series(moisture_test_list)
water_activity_test_mask = pd.Series(water_activity_test_list)

df_test['has_cannabinoid_test'] = (cannabinoid_test_mask)
df_test['has_terpenoid_test'] = (terpenoid_test_mask)
df_test['has_pesticide_test'] = (pesticide_test_mask)
df_test['has_residual_solvent_test'] = (residual_solvent_test_mask)
df_test['has_heavy_metal_test'] = (heavy_metal_test_mask)
df_test['has_microbial_test'] = (microbial_test_mask)
df_test['has_mycotoxin_test'] = (mycotoxin_test_mask)
df_test['has_foreign_material_test'] = (foreign_material_test_mask)
df_test['has_homogeneity_test'] = (homogeneity_test_mask)
df_test['has_moisture_test'] = (moisture_test_mask)
df_test['has_water_activity_test'] = (water_activity_test_mask)

In [10]:
df_canna = df_test[(df_test['has_cannabinoid_test']) == True]

In [27]:
df_canna['sample_type_id'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18])

In [113]:
df_test_test = pd.DataFrame()

for index, row in df_canna.iterrows():
    s_properties = row['properties']
    tests_ordered = {}
    tests_string = s_properties['tests_ordered'] if 'tests_ordered' in s_properties.keys() else 'None:None'
#     print(tests_string)
    if isinstance(tests_string, str):
        tests_array = tests_string.split(':')
        if len(tests_array) < 2:
#             print(tests_array, len(tests_array))
            tests_ordered['test_kind'] = ['None']
            tests_ordered['tests_ordered'] = [tests_array[0]]
        else:
            tests_ordered['test_kind'] = [tests_array[0]]
            tests_ordered['tests_ordered'] = [tests_array[1]]

#     print(tests_ordered)
    df_tests_ordered = pd.DataFrame.from_dict(tests_ordered)
#     df_tests_ordered['sample_submission_id'] = row['sample_submission_id']
    print(df_tests_ordered)



#     is_dict = type(row['coa_data']['cannabinoid_test']) is dict
#     #print('is_dict', is_dict)
#     canna_results = row['coa_data']['cannabinoid_test']['results'] if is_dict else row['coa_data']['cannabinoid_test'][0]['results']
#     print(canna_results)
#     if (type(canna_results) is not dict):
# #         not_included += 1
#         continue
#     temp_df = pd.DataFrame.from_dict(canna_results)
# #     print(canna_results)
#     print(temp_df)

# # print(df_tests_ordered)
# print(temp_df)

    test_kind                              tests_ordered
0  COMPLIANCE   POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX
  test_kind                   tests_ordered
0        QA   POTENCY, PESTICIDES, SOLVENTS
    test_kind                       tests_ordered
0  COMPLIANCE   POT, RST, HME, MTX, MBO, PES, FMT
  test_kind     tests_ordered
0        QA   POT, LCMS, GCMS
    test_kind                                   tests_ordered
0  COMPLIANCE   POT, RST, LCMS, GCMS, MBO, FMT, TER, HME, MTX
       test_kind                                      tests_ordered
0  CA COMPLIANCE   Po, Pest, Solvent, Micro, Mycotox, FM & heavy...
  test_kind tests_ordered
0        QA      POT, PES
       test_kind                                      tests_ordered
0  CA COMPLIANCE   Po, Pest, Solvent, Micro, Mycotox, FM & heavy...
       test_kind                                      tests_ordered
0  CA COMPLIANCE   Po, Pest, Solvent, Micro, Mycotox, FM & heavy...
       test_kind                  tests_ordered
0  CA 

       test_kind                  tests_ordered
0  CA COMPLIANCE   Po, Pest, Solvent, Micro, FM
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0      None       Potency
       test_kind                  tests_ordered
0  CA COMPLIANCE   Po, Pest, Solvent, Micro, FM
    test_kind                       tests_ordered
0  COMPLIANCE   POT, RST, HME, MTX, MBO, PES, FMT
       test_kind                  tests_ordered
0  CA COMPLIANCE   Po, Pest, Solvent, Micro, FM
  test_kind   tests_ordered
0        QA   POT, HME, PES
    test_kind                              tests_ordered
0  COMPLIANCE   POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX
  test_kind     tests_ordered
0        QA   POT, LCMS, GCMS
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA      POT, HME
  test_kind tests_ordered
0        QA      POT, HME
  test_kind                            tests_ordered
0        QA   POT, RST, TER, HME, MTX, MBO, PES, FMT
    test_kind   

0        QA           POT
  test_kind         tests_ordered
0        QA   POTENCY & PESTICIDE
  test_kind         tests_ordered
0        QA   POTENCY & PESTICIDE
  test_kind     tests_ordered
0        QA   POT, LCMS, GCMS
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0      None       Potency
  test_kind tests_ordered
0        QA           POT
  test_kind         tests_ordered
0        QA   POTENCY & PESTICIDE
  test_kind tests_ordered
0      None       Potency
  test_kind   tests_ordered
0        QA   POT, HME, PES
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA      POT, PES
  test_kind     tests_ordered
0        QA   POT, LCMS, GCMS
  test_kind                                   tests_ordered
0        QA   POT, RST, LCMS, GCMS, MBO, FMT, TER, HME, MTX
  test_kind tests_ordered
0        QA           POT
  test

  test_kind          tests_ordered
0        QA   POTENCY & PESTICIDES
  test_kind          tests_ordered
0        QA   POTENCY & PESTICIDES
  test_kind         tests_ordered
0        QA   POTENCY & PESTICIDE
       test_kind                                      tests_ordered
0  CA COMPLIANCE   POT, TERP, PEST, SOLV, MICRO, MYCOTOX, FM & H...
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA       POTENCY
  test_kind         tests_ordered
0        QA   POTENCY & PESTICIDE
       test_kind                                      tests_ordered
0  CA COMPLIANCE   POT, TERP, PEST, SOLV, MICRO, MYCOTOX, FM & H...
  test_kind tests_ordered
0        QA       POTENCY
    test_kind                       tests_ordered
0  COMPLIANCE   POT, RST, HME, MTX, MBO, PES, FMT
    test_kind                       tests_ordered
0  COMPLIANCE   POT, RST, HME, MTX, MBO, PES, FMT
  test_kind           tests_ordered
0      None  QA POTENCY & PESTICIDE
  test_kind tests_ordere

  test_kind                             tests_ordered
0        QA   POT, PEST, SOLV, MYCOTOX & HEAVY METALS
  test_kind                             tests_ordered
0        QA   POT, PEST, SOLV, MYCOTOX & HEAVY METALS
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA       POTENCY
    test_kind                              tests_ordered
0  COMPLIANCE   POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX
  test_kind                          tests_ordered
0        QA   POTENCY, PESTICIDES, SOLVENTS, MICRO
  test_kind                             tests_ordered
0        QA   POT, PEST, SOLV, MYCOTOX & HEAVY METALS
  test_kind tests_ordered
0        QA      POT, PES
  test_kind tests_ordered
0        QA       POTENCY
  test_kind                             tests_ordered
0        QA   POT, PEST, SOLV, MYCOTOX & HEAVY METALS
  test_kind tests_ordered
0        QA       POTENCY
  test_kind        tests_ordered
0        QA   POT, HME, MTX, PES
  test_kind tests_ordered


...

0  COMPLIANCE   POT, RST, WAT, HME, MTX, MBO, PES, MOI, FMT
       test_kind                                      tests_ordered
0  CA COMPLIANCE   RST, Pot, Pest, Micro, Mycotox, Moist, Water,...
  test_kind tests_ordered
0        QA      MOI, POT
  test_kind                                 tests_ordered
0        QA   POT, RST, WAT, HME, MTX, MBO, PES, MOI, FMT
    test_kind                                      tests_ordered
0  COMPLIANCE   MOI, POT, RST, LCMS, GCMS, MBO, FMT, TER, HME...
    test_kind                                      tests_ordered
0  COMPLIANCE   MOI, POT, RST, LCMS, GCMS, MBO, FMT, TER, HME...
       test_kind                                      tests_ordered
0  CA COMPLIANCE   RST, Pot, Pest, Micro, Mycotox, Moist, Water,...
       test_kind                                      tests_ordered
0  CA COMPLIANCE    RST, Pot, Pest, Micro, Mycotox, Moist, Water...
       test_kind                                      tests_ordered
0  CA COMPLIANCE   SOLVENT, Pot, Pes

0  COMPLIANCE   MOI, POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX...
  test_kind                                 tests_ordered
0        QA   POT, RST, WAT, HME, MTX, MBO, PES, MOI, FMT
    test_kind                                      tests_ordered
0  COMPLIANCE   MOI, POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX...
       test_kind                                      tests_ordered
0  CA COMPLIANCE   Pot, Pest, Micro, Mycotox, Moist, Water, FM &...
            test_kind                                      tests_ordered
0  RUSH CA COMPLIANCE   Pot, Pest, SOLV, Micro, Mycotox, Moist, Water...
       test_kind                                      tests_ordered
0  CA COMPLIANCE   Pot, Pest, Micro, Mycotox, Moist, Water, FM &...
    test_kind                                 tests_ordered
0  COMPLIANCE   POT, RST, WAT, HME, MTX, MBO, PES, MOI, FMT
            test_kind                                      tests_ordered
0  RUSH CA COMPLIANCE   Pot, Pest, SOLV, Micro, Mycotox, Moist, Water...
     

0  COMPLIANCE   MOI, POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX...
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA      MOI, POT
    test_kind                                 tests_ordered
0  COMPLIANCE   POT, RST, WAT, HME, MTX, MBO, PES, MOI, FMT
  test_kind tests_ordered
0        QA      MOI, POT
    test_kind                                      tests_ordered
0  COMPLIANCE   MOI, POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX...
    test_kind                                      tests_ordered
0  COMPLIANCE   MOI, POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX...
    test_kind                                 tests_ordered
0  COMPLIANCE   POT, RST, WAT, HME, MTX, MBO, PES, MOI, FMT
    test_kind                                      tests_ordered
0  COMPLIANCE   MOI, POT, GCMR, LCMS, GCMS, MBO, FMT, TER, HM...
  test_kind                                      tests_ordered
0        QA   MOI, POT, RST, LCMS, GCMS, MBO, FMT, HME, MTX...
  test_kind tests_order

0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
    test_kind                         tests_ordered
0  COMPLIANCE   POT, LCMS, GCMS, MBO, FMT, HME, MTX
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           MOI
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA    LCMS, GCMS
  test_kind tests_ordered
0        QA           POT
    test_kind tests_ordered
0  COMPLIANCE           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA              
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0       

0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
  test_kind tests_ordered
0        QA           POT
    test_kind tests_ordered
0  COMPLIANCE           POT


In [30]:
df_cannabinoid_test_data = pd.DataFrame()
not_included = 0
for index, row in df_canna.iterrows():
    is_dict = type(row['coa_data']['cannabinoid_test']) is dict
    #print('is_dict', is_dict)
    canna_results = row['coa_data']['cannabinoid_test']['results'] if is_dict else row['coa_data']['cannabinoid_test'][0]['results']
    #print(canna_results)
    if (type(canna_results) is not dict):
        not_included += 1
        continue
    temp_df = pd.DataFrame.from_dict(canna_results)
    temp_df = temp_df[temp_df.index == 'amount'].reset_index()
    temp_df['sample_submission_id'] = row['sample_submission_id']
    temp_df['sample_type_id'] = row['sample_type_id'] 
    temp_df['compliance_version'] = row['compliance_version']
    temp_df['name'] = row['name']
    temp_df['client_id']
    df_cannabinoid_test_data = df_cannabinoid_test_data.append(temp_df, ignore_index = True)
    #df_cannabinoid_test_data['sample_submission_id'] = row['sample_submission_id']
    #df_cannabinoid_test_data['sample_type_id'] = row['sample_type_id']
print(not_included)
print(df_cannabinoid_test_data.head())

/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


3
     CBC CBCA CBD CBD Per Package CBD Per Serving CBDA CBDV CBDVA    CBG  \
0  0.979    0   0             NaN             NaN    0    0     0      0   
1      0    0   0             NaN             NaN    0    0     0   1.71   
2      0    0   0             NaN             NaN    0    0     0  1.618   
3      0    0   0             NaN             NaN    0    0     0      0   
4  1.502    0   0             NaN             NaN    0    0     0  2.139   

    CBGA      ...       THCV THCVA Total CBD Total CBD Per Serving Total THC  \
0      0      ...          0     0         0                   NaN     43.63   
1      0      ...          0     0         0                   NaN    42.691   
2      0      ...          0     0         0                   NaN    52.787   
3  2.587      ...          0     0         0                   NaN   77.2286   
4      0      ...          0     0         0                   NaN   78.6016   

  Total THC Per Package Total THC Per Serving   index sample

In [50]:
df_cannabinoid_test_data[(df_cannabinoid_test_data['sample_type_id']) == 18].count()

CBC                        214
CBCA                       214
CBD                        214
CBD Per Package              0
CBD Per Serving              0
CBDA                       214
CBDV                       214
CBDVA                      214
CBG                        214
CBGA                       214
CBN                        214
Delta-8-THC                214
Delta-9-THC                214
Delta-9-THC Per Package      0
Delta-9-THC Per Serving      0
THC                          0
THC Per Serving              0
THC:CBD Ratio                0
THCA                       214
THCA Per Package             0
THCA Per Serving             0
THCV                       214
THCVA                      214
Total CBD                  214
Total CBD Per Serving        0
Total THC                  214
Total THC Per Package        0
Total THC Per Serving        0
index                      214
sample_submission_id       214
sample_type_id             214
dtype: int64

In [53]:
df_cannabinoid_test_data[(df_cannabinoid_test_data['sample_type_id']) == 1]

,CBC,CBCA,CBD,CBD Per Package,CBD Per Serving,CBDA,CBDV,CBDVA,CBG,CBGA,...,THCV,THCVA,Total CBD,Total CBD Per Serving,Total THC,Total THC Per Package,Total THC Per Serving,index,sample_submission_id,sample_type_id
0,0.979,0,0,NaN,NaN,0,0,0,0,0,...,0,0,0,NaN,43.63,NaN,NaN,amount,5374,1
1,0,0,0,NaN,NaN,0,0,0,1.71,0,...,0,0,0,NaN,42.691,NaN,NaN,amount,5673,1
2,0,0,0,NaN,NaN,0,0,0,1.618,0,...,0,0,0,NaN,52.787,NaN,NaN,amount,5674,1
3,0,0,0,NaN,NaN,0,0,0,0,2.587,...,0,0,0,NaN,77.2286,NaN,NaN,amount,6583,1
4,1.502,0,0,NaN,NaN,0,0,0,2.139,0,...,0,0,0,NaN,78.6016,NaN,NaN,amount,6535,1
5,1.485,0,0,NaN,NaN,0,0,0,2.087,0,...,0,0,0,NaN,78.4296,NaN,NaN,amount,6536,1
6,1.296,0,0,NaN,NaN,0,0,0,1.469,0,...,0,0,0,NaN,59.3596,NaN,NaN,amount,6682,1
7,1.323,0,1.115,NaN,NaN,0,0,0,1.729,0,...,0,0,1.115,NaN,57.8545,NaN,NaN,amount,6683,1
8,1.257,0,0,NaN,NaN,0,0,0,1.562,0,...,0,0,0,NaN,60.6036,NaN,NaN,amount,6748,1
9,1.227,0,0,NaN,NaN,0,0,0,2.028,0,...,0,0,0,NaN,82.7609,NaN,NaN,amount,6749,1


In [64]:
pd.Index(df_cannabinoid_test_data['sample_type_id']).value_counts()
 

1     5379
6     4957
4      988
2      414
17     288
13     269
18     214
12     156
14     155
3      149
15     132
8      126
11     111
9       23
5       18
16      10
7        4
Name: sample_type_id, dtype: int64